In [1]:
import bokeh
import json
import pandas as pd
import numpy as np
import requests
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import NumeralTickFormatter
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure
from ARCTIC import hdf5_interface
from ARCTIC import nrel_api_interface

In [2]:
location_dataframe = pd.DataFrame(columns=['location','latitude','longitude'])
location_dataframe['location']=['Ambler-Shungnak-Kobuk','Anchorage','Bethel','Chickaloon',
                       'Deering','Denali Park','Fairbanks','Fort Yukon',
                       'Galena-Koyukuk-Ruby', 'Homer','Naknek','Noatak',
                       'Noorvik','Soldotna','Valdez','Wasilla-Palmer']

location_dataframe['latitude']=[66.995834, 61.193625, 60.794938, 61.823570,
                       66.069413, 63.537277, 64.838033, 66.571563,
                       64.782991, 59.652521, 58.728349, 67.570921,
                       66.836039, 60.486370, 61.128663, 61.582242]


location_dataframe['longitude']=[ -157.377096, -149.694974, -161.770716, -148.450442,
                        -162.766760,  -150.985453, -147.668970,  -145.250173,
                        -156.744933, -151.536496, -157.017444, -162.967490,
                         -161.041913, -151.060702, -146.353366, -149.441001]


location_dataframe

,location,latitude,longitude
0,Ambler-Shungnak-Kobuk,66.995834,-157.377096
1,Anchorage,61.193625,-149.694974
2,Bethel,60.794938,-161.770716
3,Chickaloon,61.823570,-148.450442
4,Deering,66.069413,-162.766760
5,Denali Park,63.537277,-150.985453
6,Fairbanks,64.838033,-147.668970
7,Fort Yukon,66.571563,-145.250173
8,Galena-Koyukuk-Ruby,64.782991,-156.744933
9,Homer,59.652521,-151.536496


In [52]:
def tilt_angle_plot_generation(location_dataframe): 
    """This function takes in a dataframe that contains latitudes and longitudes for a number of 
    locations and generates interactive Bokeh plots showing the variation of monthly production 
    with changing tilt angles."""
    #The below list is sufficiently granular to cover most situations.
    tilt_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]

    #Walk through each row in the location dataframe, calling PVwatts and plotting results.
    for j in range(len(location_dataframe.index)):
        print("Data for " + str(location_dataframe['location'][j]) + " is being calculated")
        nrel_long_tilt = []
        for i in range(len(tilt_list)):
            json_response, new_dataframe = nrel_api_interface.call_pvwatts(latitude = location_dataframe['latitude'][j], 
                                                                                     longitude = location_dataframe['longitude'][j], 
                                                                                     tilt = tilt_list[i], dataset = 'tmy3')
            nrel_long_tilt.append(new_dataframe)
        tilt_response_dataframe = pd.DataFrame(columns = tilt_list)
        for i, tilt in enumerate(tilt_list):
            tilt_response_dataframe[tilt] = nrel_long_tilt[i]['ac_monthly']/4

        #The below is all of the data for the plotting components.
        #This adjusts the name of the saved file, so it's specific to each location.
        output_file("{}_monthly_production_varying_tilts.html".format(location_dataframe['location'][j]))
        #Set up a month proxy
        x = np.arange(1,13)

        #Tell the plot where to look for the data. The extra specifications of y values
        #enable the plot to be interactive.
        source = ColumnDataSource(data=dict(x=x, y=tilt_response_dataframe[5],
                                            tilt_5_degrees=tilt_response_dataframe[5], tilt_10_degrees=tilt_response_dataframe[10],
                                            tilt_15_degrees=tilt_response_dataframe[15], tilt_20_degrees=tilt_response_dataframe[20],
                                            tilt_25_degrees=tilt_response_dataframe[25], tilt_30_degrees=tilt_response_dataframe[30],
                                            tilt_35_degrees=tilt_response_dataframe[35], tilt_40_degrees=tilt_response_dataframe[40],
                                            tilt_45_degrees=tilt_response_dataframe[45], tilt_50_degrees=tilt_response_dataframe[50],
                                            tilt_55_degrees=tilt_response_dataframe[55], tilt_60_degrees=tilt_response_dataframe[60],
                                            tilt_65_degrees=tilt_response_dataframe[65], tilt_70_degrees=tilt_response_dataframe[70],
                                            tilt_75_degrees=tilt_response_dataframe[75], tilt_80_degrees=tilt_response_dataframe[80],
                                            tilt_85_degrees=tilt_response_dataframe[85], tilt_90_degrees=tilt_response_dataframe[90],
                                           ))
        #Plot specifications
        plot = figure(x_axis_label='Month', y_axis_label='Normalized Monthly Production (kWh/kW)', plot_height=400)
        plot.line(x='x', y='y', source=source)
        plot.title.text = "Annual Production at Varying Tilt Angles"
        plot.title.align = "center"
        plot.title.text_font = "times"
        plot.title.text_font_style = "italic"
        plot.title.text_font_size = '15pt'
        #This line is what connects the changing dropdown menu with the data that is displayed.
        select = Select(value='foo', options=['tilt_5_degrees', 'tilt_10_degrees','tilt_15_degrees',
                                             'tilt_20_degrees','tilt_25_degrees','tilt_30_degrees',
                                             'tilt_35_degrees','tilt_40_degrees','tilt_45_degrees',
                                             'tilt_50_degrees','tilt_55_degrees','tilt_60_degrees',
                                             'tilt_65_degrees','tilt_70_degrees','tilt_75_degrees',
                                             'tilt_80_degrees','tilt_85_degrees','tilt_90_degrees'])
        #javascript that actually makes the changes possible.
        select.js_on_change('value', CustomJS(args=dict(source=source, select=select), code="""
            // make a shallow copy of the current data dict
            const new_data = Object.assign({}, source.data)

            // update the y column in the new data dict from the appropriate other column
            new_data.y = source.data[select.value]

            // set the new data on source, BokehJS will pick this up automatically
            source.data = new_data
        """))

        show(column(plot, select))


In [53]:
tilt_angle_plot_generation(location_dataframe)

Data for Ambler-Shungnak-Kobuk is being calculated
Data for Anchorage is being calculated


KeyboardInterrupt: 

In [11]:
#Begin tilt vs annual production graphic generation.
def annual_tilt_angle_plot_generation(location_dataframe):
    """This function plots the annual generation predicted by PVWatts for TMY2 and TMY3 datasets
    at a variety of tilt angles. It operates dynamically, and prints for all locations in the 
    passed location dataframe."""
    #The below list is sufficiently granular to cover most situations.
    tilt_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
    
    #Walk through each row in the location dataframe, calling PVwatts and plotting results.
    for j in range(len(location_dataframe.index)):
        print("Data for " + str(location_dataframe['location'][j]) + " is being calculated")
        #If the response from PVWatts has an error, flip the zero to a one, and don't plot.
        tmy2_response = np.zeros(len(tilt_list))
        tmy3_response = np.zeros(len(tilt_list))
        annual_production_tmy2 = []
        annual_production_tmy3 = []
        #Calculate all of the tilt information for TMY3 and TMY3. 
        for i in range(len(tilt_list)):
            json_response, new_dataframe = nrel_api_interface.call_pvwatts(latitude = location_dataframe['latitude'][j], 
                                                                                     longitude = location_dataframe['longitude'][j], 
                                                                                     tilt = tilt_list[i], dataset = 'tmy3')
            if json_response['errors'] != []:
                tmy3_response[i] = 1
            else:
                annual_production_tmy3.append(new_dataframe['ac_annual'][2]/4)
            
            #repeat the above with the TMY2 dataset
            json_response, new_dataframe = nrel_api_interface.call_pvwatts(latitude = location_dataframe['latitude'][j], 
                                                                                 longitude = location_dataframe['longitude'][j], 
                                                                                 tilt = tilt_list[i], dataset = 'tmy2')
            if json_response['errors'] != []:
                tmy2_response[i] = 1
            else:    
                annual_production_tmy2.append(new_dataframe['ac_annual'][2]/4)
        
        #Save the file
        output_file("{}_annual_production_varying_tilts.html".format(location_dataframe['location'][j]))
        p = figure( x_axis_label='Tilts', y_axis_label='Annual Production (kWh)',plot_width=500, plot_height=250)

        # add a line renderer
        #Check to see if there is TMY2 or TMY3 data for this location. 
        if sum(tmy2_response) == 0:
            p.line(tilt_list, annual_production_tmy2, line_width=2, color='red', legend='TMY2')
        else:
            print("No TMY2 data was available for this location.")
        if sum(tmy3_response) == 0:    
            p.line(tilt_list, annual_production_tmy3, line_width=2, color='blue', legend='TMY3')
        else:
            print("No TMY3 data was available for this location")

        p.xaxis.ticker = [10,20,30,40,50,60,70,80,90]
        p.title.text = "Annual Production at Varying Tilts"
        p.title.align = "center"
        p.title.text_font = "times"
        p.title.text_font_style = "italic"
        p.title.text_font_size = '12pt'

        show(p)           

In [12]:
annual_tilt_angle_plot_generation(location_dataframe)

Data for Ambler-Shungnak-Kobuk is being calculated
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.995834 lon=-157.377096']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.995834 lon=-157.377096']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong 

Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.571563 lon=-145.250173']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.571563 lon=-145.250173']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning!

Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=64.782991 lon=-156.744933']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=64.782991 lon=-156.744933']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning!

Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=59.652521 lon=-151.536496']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=59.652521 lon=-151.536496']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning!

In [18]:
def annual_production_loss_plot_generation(location_dataframe):
    """This function plots the percentage loss of production for tilt angles that are not
    in alignment with the optimal tilt angle."""    
    #The below list is sufficiently granular to cover most situations.
    tilt_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
    
    #Walk through each row in the location dataframe, calling PVwatts and plotting results.
    for j in range(len(location_dataframe.index)):
        print("Data for " + str(location_dataframe['location'][j]) + " is being calculated")
        #If the response from PVWatts has an error, flip the zero to a one, and don't plot.
        tmy2_response = np.zeros(len(tilt_list))
        tmy3_response = np.zeros(len(tilt_list))
        annual_production_tmy2 = []
        annual_production_tmy3 = []
        #Calculate all of the tilt information for TMY3 and TMY3. 
        for i in range(len(tilt_list)):
            json_response, new_dataframe = nrel_api_interface.call_pvwatts(latitude = location_dataframe['latitude'][j], 
                                                                                     longitude = location_dataframe['longitude'][j], 
                                                                                     tilt = tilt_list[i], dataset = 'tmy3')
            if json_response['errors'] != []:
                tmy3_response[i] = 1
            else:
                annual_production_tmy3.append(new_dataframe['ac_annual'][2]/4)
            
            #repeat the above with the TMY2 dataset
            json_response, new_dataframe = nrel_api_interface.call_pvwatts(latitude = location_dataframe['latitude'][j], 
                                                                                 longitude = location_dataframe['longitude'][j], 
                                                                                 tilt = tilt_list[i], dataset = 'tmy2')
            if json_response['errors'] != []:
                tmy2_response[i] = 1
            else:    
                annual_production_tmy2.append(new_dataframe['ac_annual'][2]/4)
        
        if sum(tmy2_response) == 0:
            dict_tmy2 = {'Tilts':tilt_list,'Annual_production':annual_production_tmy2}
            df_tmy2 = pd.DataFrame(dict_tmy2)
            #Then find out the max production row
            max_tilt_tmy2 = int(df_tmy2[['Annual_production']].idxmax().values)
            #Then calculate the other tilts' percent loss compared with the max annual production
            lose_tmy2 = []
            for index, row in df_tmy2.iterrows():
                tilt_loss = 1- row['Annual_production']/df_tmy2['Annual_production'][max_tilt_tmy2]
                lose_tmy2.append(tilt_loss)
            df_tmy2['loss']=lose_tmy2

        else:
            print("There is no TMY2 weather station data at this location.")
            
        #Repeat for TMY3 data.
        if sum(tmy3_response) == 0:
            dict_tmy3 = {'Tilts':tilt_list,'Annual_production':annual_production_tmy3}
            df_tmy3 = pd.DataFrame(dict_tmy3)
            max_tilt_tmy3 = int(df_tmy3[['Annual_production']].idxmax().values)
            lose_tmy3 = []
            for index, row in df_tmy3.iterrows():
                tilt_loss = 1- row['Annual_production']/df_tmy3['Annual_production'][max_tilt_tmy3]
                lose_tmy3.append(tilt_loss)
            df_tmy3['loss']=lose_tmy3   

        #Save the file
        output_file("{}_annual_production_loss_tilts.html".format(location_dataframe['location'][j]))

        p = figure(x_axis_label='Tilts', y_axis_label='loss (%)',plot_width=500, plot_height=250)

        # add a line renderer
        if sum(tmy2_response) == 0:
            p.line(tilt_list, df_tmy2['loss'], line_width=2,color='red',legend="TMY2")
        if sum(tmy3_response) == 0:
            p.line(tilt_list, df_tmy3['loss'],line_width=2,color='blue',legend="TMY3")
        p.xaxis.ticker = [10,20,30,40,50,60,70,80,90]
        p.yaxis.formatter = NumeralTickFormatter(format='0 %')
        p.title.text = "Annual Production Loss with Varying Tilts"
        p.title.align = "center"
        p.title.text_font = "times"
        p.title.text_font_style = "italic"
        p.title.text_font_size = '12pt'

        show(p)    
        

In [19]:
annual_production_loss_plot_generation(location_dataframe)

Data for Ambler-Shungnak-Kobuk is being calculated
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.995834 lon=-157.377096']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong with your inputs (out of bounds possibly).
Warning! An error occured when calling PVWatts. The error was: 
['No climate data found with dataset=tmy2 for location specified: lat=66.995834 lon=-157.377096']
Please correct the error and call this function again.
Warning! PVWatts had an error 422. This means either the solar dataset does not exist for 
your intended location, or that something was wrong 

AssertionError: Error: 429